In [23]:
import cv2
import numpy as np
import onnxruntime as ort

In [24]:
def img_preprocessing(img):
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    for i in range(3):
        img[:, :, i] = (img[:, :, i] - mean[i]) / std[i]
    img = np.transpose(img, (2, 0, 1))
    return img

In [ ]:
def predict_on_cammera(onnx_model_path,n_fps_pred=16,n_frames_pred=15):
    #loader model
    
    session = ort.InferenceSession(onnx_model_path)
    input_name=session.get_inputs()[0].name
    output_name=session.get_outputs()[0].name

    cap = cv2.VideoCapture(0)
    fps=cap.get(cv2.CAP_PROP_FPS)
    interval=int(fps/n_fps_pred)
    X=[]
    violent_frames=[]
    tmp_violent_frames=[]
    counts=0
    while True:
        counts+=1
        if(counts%interval==0):
            counts=0
            ret,frame=cap.read()
            cv2.imshow('cammera',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if not ret:
                break
            processed_frame=img_preprocessing(frame)
            X.append(processed_frame)
            tmp_violent_frames.append(frame)
        input_data=np.array([X[-n_frames_pred:].copy()],dtype=np.float32)
        if(len(input_data[0])!=n_frames_pred):
            pass
        else:
            X=X[-n_frames_pred:]
            tmp_violent_frames=tmp_violent_frames[-n_frames_pred:]
            outputs = session.run([output_name], {input_name: input_data})
            y_preds=np.argmax(outputs[0],axis=1)[0]
            # print(outputs[0][0])
            if(y_preds==1):
                if(len(violent_frames)>=n_frames_pred):
                    violent_frames.extend(tmp_violent_frames[-1])
                else:
                    violent_frames.extend(tmp_violent_frames)
                print(y_preds)
    cap.release()
    cv2.destroyAllWindows()

    return violent_frames
        

In [ ]:
violentFrames=predict_on_cammera(onnx_model_path=r'D:\DS\ViolenceDetection\notebooks\model_quantized.onnx',n_fps_pred=16,n_frames_pred=10)

1
1
1
1
1
1
1


In [27]:
np.save('array.npy', violentFrames)

In [28]:
np.array(violentFrames).shape

(84, 480, 640, 3)

In [29]:
import matplotlib.pyplot as plt

In [ ]:
for i in range(len(violentFrames)):
    # Show each frame
    # if(i>=12):

    cv2.imshow('Video Frame', violentFrames[min(len(violentFrames)-1,i+11)])
    # else:
    #     cv2.imshow('Video Frame', violentFrames[i])
    
    # Wait for 25 ms between frames (40 FPS). Adjust as needed.
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break  # Press 'q' to exit early

cv2.destroyAllWindows()

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "d:\Anaconda\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "d:\Anaconda\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "d:\Anaconda\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "d:\Anaconda\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was f

: 